In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from pprint import pprint
%matplotlib inline
import json
import time
import tqdm

In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 10000

This section focuses on flattening out the player loadout column and creating one single dataframe

In [3]:
king_df = pd.read_csv('mega_df.csv', index_col = 0)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [80]:
king_df.head(5)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,teamCount,rankedTeams,draw,privateMatch,playerStats.kills,playerStats.medalXp,playerStats.objectiveLastStandKill,playerStats.matchXp,playerStats.scoreXp,playerStats.wallBangs,playerStats.score,playerStats.totalXp,playerStats.headshots,playerStats.assists,playerStats.challengeXp,playerStats.rank,playerStats.scorePerMinute,playerStats.distanceTraveled,playerStats.teamSurvivalTime,playerStats.deaths,playerStats.objectiveMunitionsBoxTeammateUsed,playerStats.kdRatio,playerStats.objectiveBrDownEnemyCircle1,playerStats.bonusXp,playerStats.objectiveBrKioskBuy,playerStats.gulagDeaths,playerStats.timePlayed,playerStats.executions,playerStats.gulagKills,playerStats.nearmisses,playerStats.objectiveBrCacheOpen,playerStats.percentTimeMoving,playerStats.miscXp,playerStats.longestStreak,playerStats.teamPlacement,playerStats.damageDone,playerStats.damageTaken,player.team,player.rank,player.username,player.uno,player.clantag,player.brMissionStats.missionsComplete,player.brMissionStats.totalMissionXpEarned,player.brMissionStats.totalMissionWeaponXpEarned,player.loadout,playerStats.objectiveTeamWiped,playerStats.objectiveBrMissionPickupTablet,player.brMissionStats.missionStatsByType.timedrun.weaponXp,player.brMissionStats.missionStatsByType.timedrun.xp,player.brMissionStats.missionStatsByType.timedrun.count,playerStats.objectiveBrDownEnemyCircle3,playerStats.objectiveBrDownEnemyCircle2,playerStats.objectiveReviver,player.brMissionStats.missionStatsByType.vip.weaponXp,player.brMissionStats.missionStatsByType.vip.xp,player.brMissionStats.missionStatsByType.vip.count,playerStats.objectiveAssistDecoy,player.brMissionStats.missionStatsByType.assassination.weaponXp,player.brMissionStats.missionStatsByType.assassination.xp,player.brMissionStats.missionStatsByType.assassination.count,playerStats.objectiveDestroyedVehicleLight,playerStats.objectiveBrDownEnemyCircle4,playerStats.objectiveBrDownEnemyCircle6,player.brMissionStats.missionStatsByType.scavenger.weaponXp,player.brMissionStats.missionStatsByType.scavenger.xp,player.brMissionStats.missionStatsByType.scavenger.count,player.brMissionStats.missionStatsByType.domination.weaponXp,player.brMissionStats.missionStatsByType.domination.xp,player.brMissionStats.missionStatsByType.domination.count,playerStats.objectiveDestroyedEquipment,playerStats.objectiveMedalScoreSsKillPrecisionAirstrike,playerStats.objectiveDestroyedVehicleHeavy,playerStats.objectiveTrophyDefense,player.awards.mode_x_eliminate,player.awards.headshot,playerStats.objectiveBrDownEnemyCircle5,player.brMissionStats.missionStatsByType.blueprintextract.weaponXp,player.brMissionStats.missionStatsByType.blueprintextract.xp,player.brMissionStats.missionStatsByType.blueprintextract.count,playerStats.objectivePerkMarkedTarget,player.awards.one_shot_kill,player.awards.pointblank,player.awards.longshot,player.awards.gun_butt,player.awards.firstblood,player.awards.save_teammate,playerStats.objectiveDestroyedVehicleMedium,player.awards.revenge,player.awards.triple,player.awards.kill_jumper,player.awards.one_shot_two_kills,player.awards.streak_5,player.awards.streak_10,playerStats.objectiveShieldDamage,playerStats.objectiveMedalScoreSsKillTomaStrike,playerStats.objectiveHack,playerStats.objectiveBinocularsMarked,player.awards.double,player.awards.fist_kill,player.awards.low_health_kill,player.awards.throwingknife_kill,player.awards.posthumous,playerStats.objectiveBrLootChopperBoxOpen,player.awards.stunned_kill,playerStats.objectiveManualFlareMissileRedirect,player.awards.avenger,playerStats.objectiveMedalScoreKillSsRadarDrone,player.awards.explosive_stick,player.awards.air_to_air_kill,player.awards.streak_15,player.awards.mode_x_last_alive,playerStats.objectivePlunderCashBloodMoney,playerStats.objectiveBrC130BoxOpen,playerStats.objectiveEmpedVehicle,playerStats.objectiveEmpedPlayer,player.awards.slide_kill,player.awards.match_complete,player.awards.match_co

In [7]:
# Extracting the loadout column so that it can be focused on
loadout = king_df['player.loadout']

In [8]:
# this function removes the lists wrapping the dictionaries
def converter_x(x):
    try:
        return eval(x)[0]
    except:
        return np.nan

In [9]:
loadout2 = loadout.apply(converter_x)

In [12]:
# filling all NAs with an empty dictionary so that a weapons dataframe can be created
loadout2 = loadout2.fillna(1).map(lambda x: {} if x == 1 else x)

In [13]:
loadout3 = pd.DataFrame(loadout2.values.tolist())

In [81]:
loadout3.head(5)

,primaryWeapon,secondaryWeapon,perks,extraPerks,killstreaks,tactical,lethal
0,"{'name': 'iw8_lm_lima86', 'label': None, 'imag...","{'name': 'iw8_pi_t9revolver', 'label': None, '...","[{'name': 'specialty_null', 'label': None, 'im...","[{'name': 'null', 'label': None, 'image': None...","[{'name': 'none', 'label': None}, {'name': 'no...","{'name': 'equip_concussion', 'label': None, 'i...","{'name': 'equip_at_mine', 'label': None, 'imag..."
1,"{'name': 'iw8_lm_lima86', 'label': None, 'imag...","{'name': 'iw8_pi_t9revolver', 'label': None, '...","[{'name': 'specialty_null', 'label': None, 'im...","[{'name': 'null', 'label': None, 'image': None...","[{'name': 'none', 'label': None}, {'name': 'no...","{'name': 'equip_concussion', 'label': None, 'i...","{'name': 'equip_at_mine', 'label': None, 'imag..."
2,"{'name': 'iw8_lm_lima86', 'label': None, 'imag...","{'name': 'iw8_pi_t9revolver', 'label': None, '...","[{'name': 'specialty_null', 'label': None, 'im...","[{'name': 'null', 'label': None, 'image': None...","[{'name': 'none', 'label': None}, {'name': 'no...","{'name': 'equip_concussion', 'label': None, 'i...","{'name': 'equip_at_mine', 'label': None, 'imag..."
3,"{'name': 'iw8_sh_t9pump', 'label': None, 'imag...","{'name': 'iw8_pi_papa320', 'label': None, 'ima...","[{'name': 'specialty_null', 'label': None, 'im...","[{'name': 'null', 'label': None, 'image': None...","[{'name': 'none', 'label': None}, {'name': 'no...","{'name': 'equip_concussion', 'label': None, 'i...","{'name': 'equip_molotov', 'label': None, 'imag..."
4,"{'name': 'iw8_sh_t9pump', 'label': None, 'imag...","{'name': 'iw8_pi_papa320', 'label': None, 'ima...","[{'name': 'specialty_null', 'label': None, 'im...","[{'name': 'null', 'label': None, 'image': None...","[{'name': 'none', 'label': None}, {'name': 'no...","{'name': 'equip_concussion', 'label': None, 'i...","{'name': 'equip_molotov', 'label': None, 'imag..."


In [15]:
#  This function fills the NAs with empty dictionaries so that the nested dictionaries within each column of loadout3
# can be converted into individual dataframes: Primary weapon, Secondary Wepon, Perks, Extra Perks, Killstreaks, Tactical and Lethal
def turnover(z):
    t = z.fillna(1).map(lambda x: {} if x == 1 else x)
    return t

In [31]:
loadout4 = loadout3.apply(turnover)

In [ ]:
# The creation of seperate dataframes for each column

In [32]:
primary_weapon = pd.DataFrame(loadout4.primaryWeapon.values.tolist()).apply(turnover)

In [33]:
secondary_weapon = pd.DataFrame(loadout4.secondaryWeapon.values.tolist()).apply(turnover)

In [34]:
tactical = pd.DataFrame(loadout4.tactical.values.tolist()).apply(turnover)

In [35]:
lethal = pd.DataFrame(loadout4.lethal.values.tolist()).apply(turnover)

In [36]:
perks = pd.DataFrame(loadout4.perks.values.tolist()).apply(turnover)

In [37]:
extraperks = pd.DataFrame(loadout4.extraPerks.values.tolist()).apply(turnover)

In [38]:
killstreaks = pd.DataFrame(loadout4.killstreaks.values.tolist()).apply(turnover)

In [ ]:
# Further flattening the respective dataframes

In [82]:
killstreaks.head()

,0,1,2
0,"{'name': 'none', 'label': None}","{'name': 'none', 'label': None}","{'name': 'none', 'label': None}"
1,"{'name': 'none', 'label': None}","{'name': 'none', 'label': None}","{'name': 'none', 'label': None}"
2,"{'name': 'none', 'label': None}","{'name': 'none', 'label': None}","{'name': 'none', 'label': None}"
3,"{'name': 'none', 'label': None}","{'name': 'none', 'label': None}","{'name': 'none', 'label': None}"
4,"{'name': 'none', 'label': None}","{'name': 'none', 'label': None}","{'name': 'none', 'label': None}"


In [40]:
# Killstreaks

kills_dfs = []
for i in range(3):
    ks = pd.DataFrame(killstreaks[i].values.tolist(), columns = ['name'])
    ks.columns = [f'killstreak_{i+1}']
    kills_dfs.append(ks)

killstreak_dfs = pd.concat(kills_dfs, axis = 1)    


In [43]:
# Perks

p_dfs = []
for i in range(3):
    ks = pd.DataFrame(perks[i].values.tolist(), columns = ['name'])
    ks.columns = [f'perk_{i+1}']
    p_dfs.append(ks)

perks_dfs = pd.concat(p_dfs, axis = 1)   

In [45]:
#  Extra Perks

ep_dfs = []
for i in range(3):
    ks = pd.DataFrame(extraperks[i].values.tolist(), columns = ['name'])
    ks.columns = [f'extraperk_{i+1}']
    ep_dfs.append(ks)

extraperks_dfs = pd.concat(ep_dfs, axis = 1)   

In [ ]:
# Primary weapon attachments

In [49]:
attachments = pd.DataFrame(primary_weapon.attachments.values.tolist()).apply(turnover)

In [50]:
attach = []
for i in range(5):
    ks = pd.DataFrame(attachments[i].values.tolist(), columns = ['name'])
    ks.columns = [f'attachment_{i+1}']
    attach.append(ks)

attachments_dfs = pd.concat(attach, axis = 1)   

In [ ]:
# Secondary weapon attachments

In [53]:
attachments_secondary = pd.DataFrame(secondary_weapon.attachments.values.tolist()).apply(turnover)

In [54]:
attach = []
for i in range(5):
    ks = pd.DataFrame(attachments_secondary[i].values.tolist(), columns = ['name'])
    ks.columns = [f'sec_attachment_{i+1}']
    attach.append(ks)

attachments_secondary_dfs = pd.concat(attach, axis = 1)

In [ ]:
# Extracting the Grenades and Weapons into one dataframe

In [ ]:
a = pd.DataFrame(primary_weapon.name)
b = pd.DataFrame(secondary_weapon.name)
c = pd.DataFrame(lethal.name)
d = pd.DataFrame(tactical.name)

In [61]:
weaPons = pd.concat([a,b,c,d], axis = 1)

In [62]:
weaPons.columns = ['primary','secondary', 'lethal', 'tactical']

In [63]:
weaPons.tail()

,primary,secondary,lethal,tactical
148849,iw8_lm_pkilo,iw8_sn_crossbow,equip_molotov,equip_flash
148850,iw8_ar_mcharlie,iw8_sh_charlie725,equip_frag,equip_concussion
148851,iw8_ar_mcharlie,iw8_sh_charlie725,equip_frag,equip_concussion
148852,iw8_lm_pkilo,iw8_sn_crossbow,equip_molotov,equip_flash
148853,iw8_ar_mcharlie,iw8_sh_charlie725,equip_frag,equip_concussion


In [64]:
# Creating one dataframe for the flattened player loadout column
prince_df = pd.concat([weaPons,attachments_dfs,attachments_secondary_dfs,perks_dfs,extraperks_dfs,killstreak_dfs], axis =1)

In [66]:
# Adding the new player loadout columns to the main dataframe
for col in prince_df:
    king_df[col] = prince_df[col]

In [67]:
king_df.shape

(148854, 180)

In [68]:
emperor_df = king_df.drop(columns = ['player.loadout'])

In [69]:
# emperor_df .to_csv('emperor_df.csv')

In [84]:
emperor_df.head(5)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,teamCount,rankedTeams,draw,privateMatch,playerStats.kills,playerStats.medalXp,playerStats.objectiveLastStandKill,playerStats.matchXp,playerStats.scoreXp,playerStats.wallBangs,playerStats.score,playerStats.totalXp,playerStats.headshots,playerStats.assists,playerStats.challengeXp,playerStats.rank,playerStats.scorePerMinute,playerStats.distanceTraveled,playerStats.teamSurvivalTime,playerStats.deaths,playerStats.objectiveMunitionsBoxTeammateUsed,playerStats.kdRatio,playerStats.objectiveBrDownEnemyCircle1,playerStats.bonusXp,playerStats.objectiveBrKioskBuy,playerStats.gulagDeaths,playerStats.timePlayed,playerStats.executions,playerStats.gulagKills,playerStats.nearmisses,playerStats.objectiveBrCacheOpen,playerStats.percentTimeMoving,playerStats.miscXp,playerStats.longestStreak,playerStats.teamPlacement,playerStats.damageDone,playerStats.damageTaken,player.team,player.rank,player.username,player.uno,player.clantag,player.brMissionStats.missionsComplete,player.brMissionStats.totalMissionXpEarned,player.brMissionStats.totalMissionWeaponXpEarned,playerStats.objectiveTeamWiped,playerStats.objectiveBrMissionPickupTablet,player.brMissionStats.missionStatsByType.timedrun.weaponXp,player.brMissionStats.missionStatsByType.timedrun.xp,player.brMissionStats.missionStatsByType.timedrun.count,playerStats.objectiveBrDownEnemyCircle3,playerStats.objectiveBrDownEnemyCircle2,playerStats.objectiveReviver,player.brMissionStats.missionStatsByType.vip.weaponXp,player.brMissionStats.missionStatsByType.vip.xp,player.brMissionStats.missionStatsByType.vip.count,playerStats.objectiveAssistDecoy,player.brMissionStats.missionStatsByType.assassination.weaponXp,player.brMissionStats.missionStatsByType.assassination.xp,player.brMissionStats.missionStatsByType.assassination.count,playerStats.objectiveDestroyedVehicleLight,playerStats.objectiveBrDownEnemyCircle4,playerStats.objectiveBrDownEnemyCircle6,player.brMissionStats.missionStatsByType.scavenger.weaponXp,player.brMissionStats.missionStatsByType.scavenger.xp,player.brMissionStats.missionStatsByType.scavenger.count,player.brMissionStats.missionStatsByType.domination.weaponXp,player.brMissionStats.missionStatsByType.domination.xp,player.brMissionStats.missionStatsByType.domination.count,playerStats.objectiveDestroyedEquipment,playerStats.objectiveMedalScoreSsKillPrecisionAirstrike,playerStats.objectiveDestroyedVehicleHeavy,playerStats.objectiveTrophyDefense,player.awards.mode_x_eliminate,player.awards.headshot,playerStats.objectiveBrDownEnemyCircle5,player.brMissionStats.missionStatsByType.blueprintextract.weaponXp,player.brMissionStats.missionStatsByType.blueprintextract.xp,player.brMissionStats.missionStatsByType.blueprintextract.count,playerStats.objectivePerkMarkedTarget,player.awards.one_shot_kill,player.awards.pointblank,player.awards.longshot,player.awards.gun_butt,player.awards.firstblood,player.awards.save_teammate,playerStats.objectiveDestroyedVehicleMedium,player.awards.revenge,player.awards.triple,player.awards.kill_jumper,player.awards.one_shot_two_kills,player.awards.streak_5,player.awards.streak_10,playerStats.objectiveShieldDamage,playerStats.objectiveMedalScoreSsKillTomaStrike,playerStats.objectiveHack,playerStats.objectiveBinocularsMarked,player.awards.double,player.awards.fist_kill,player.awards.low_health_kill,player.awards.throwingknife_kill,player.awards.posthumous,playerStats.objectiveBrLootChopperBoxOpen,player.awards.stunned_kill,playerStats.objectiveManualFlareMissileRedirect,player.awards.avenger,playerStats.objectiveMedalScoreKillSsRadarDrone,player.awards.explosive_stick,player.awards.air_to_air_kill,player.awards.streak_15,player.awards.mode_x_last_alive,playerStats.objectivePlunderCashBloodMoney,playerStats.objectiveBrC130BoxOpen,playerStats.objectiveEmpedVehicle,playerStats.objectiveEmpedPlayer,player.awards.slide_kill,player.awards.match_complete,player.awards.match_complete_win,play